# Import libraries and model hyperparameters

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import argparse
import random
random.seed(1)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, LSTM, Dropout, Flatten, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split

2024-08-12 06:13:46.223453: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-12 06:13:46.449217: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-12 06:13:46.495211: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-08-12 06:13:46.495225: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudar

In [27]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Define model hyperparameters
parser = argparse.ArgumentParser(description='CNN-LSTM Base Model')
parser.add_argument('--input_size', type=int, default=4) #modify from 7 due to reducing outdoor temp/RH and NV
parser.add_argument('--batch_size', type=int, default=128)
parser.add_argument('--num_epochs', type=int, default=100)
parser.add_argument('--learning_rate', type=float, default=0.001)
parser.add_argument('--input_features', type=list, default=['Indoor Temp',
                                                            'Indoor Humidity',
                                                            'Air Velocity',
                                                            'Globe Temperature'])

parser.add_argument('--experiment', type=str, default='mode_random') 
# choose either 'condition_random', 'mode_random', 'mode_al'
args = parser.parse_args(args=[])

mode_mapping = {'AC':0, 'NV':1}
thermalpref_mapping = {'No Change':0, 'Warmer':1, 'Cooler':2}
thermalacc_mapping = {'Acceptable':0, 'Unacceptable':1}
airpref_mapping = {'No Change':0, 'More':1, 'Less':2}

In [11]:
# load training/eval/test data from ASHRAE dataset
thermalpref_df = pd.read_csv('../data/ashrae_thermalpref_sampled_data.csv')
thermalacc_df = pd.read_csv('../data/ashrae_thermalacc_sampled_data.csv')
airpref_df = pd.read_csv('../data/ashrae_airpref_sampled_data.csv')

# # load training data from BCA dataset
# if args.experiment == 'condition_random':
#     thermalpref_train = pd.read_csv('../data/bca_thermalpref_train_condition_random_data.csv')
#     thermalacc_train = pd.read_csv('../data/bca_thermalacc_train_condition_random_data.csv')
#     airpref_train = pd.read_csv('../data/bca_airpref_train_condition_random_data.csv')
# elif args.experiment == 'mode_random':
#     thermalpref_train = pd.read_csv('../data/bca_thermalpref_train_mode_random_data.csv')
#     thermalacc_train = pd.read_csv('../data/bca_thermalacc_train_mode_random_data.csv')
#     airpref_train = pd.read_csv('../data/bca_airpref_train_mode_random_data.csv')
# elif args.experiment == 'mode_al':
#     thermalpref_train = pd.read_csv('../data/bca_thermalpref_train_mode_al_data.csv')
#     thermalacc_train = pd.read_csv('../data/bca_thermalacc_train_mode_al_data.csv')
#     airpref_train = pd.read_csv('../data/bca_airpref_train_mode_al_data.csv')
# else:
#     raise ValueError(f"Experiment {args.experiment} is not supported.")

# # load test data from BCA dataset
# thermalpref_test = pd.read_csv('../data/bca_thermalpref_test_data.csv')
# thermalacc_test = pd.read_csv('../data/bca_thermalacc_test_data.csv')
# airpref_test = pd.read_csv('../data/bca_airpref_test_data.csv')

In [19]:
def filter_and_split_data(df, target_col, test_size=0.2, eval_size=0.1):
    """
    Splits the dataset into train, evaluation, and test sets.

    Parameters:
        df: The entire dataframe containing features and target column.
        target_col: The name of the target column in the dataframe.
        test_size: The proportion of the dataset to include in the test split.
        eval_size: The proportion of the dataset to include in the eval split.

    Returns:
        train_df, eval_df, test_df: Three dataframes for training, evaluation, and testing.
    """

    # Filter the dataframe for rows where 'Mode' is 'AC'
    filtered_df = df[df['Mode'] == 'AC']
    
    # Split df into train+eval and test
    train_eval_df, test_df = train_test_split(df, test_size=test_size, stratify=df[target_col])

    # Further split train+eval into train and eval
    train_df, eval_df = train_test_split(train_eval_df, test_size=eval_size / (1 - test_size), stratify=train_eval_df[target_col])

    return train_df, eval_df, test_df

In [20]:
# Filter by 'Mode' == 'AC' and split the dataframes
thermalpref_train, thermalpref_eval, thermalpref_test = filter_and_split_data(thermalpref_df, target_col='Thermal Preference')
thermalacc_train, thermalacc_eval, thermalacc_test = filter_and_split_data(thermalacc_df, target_col='Thermal Acceptability')
airpref_train, airpref_eval, airpref_test = filter_and_split_data(airpref_df, target_col='Air Movement Preference')

In [21]:
# perform label mapping
thermalpref_train['Thermal Preference'] = thermalpref_train['Thermal Preference'].apply(lambda x: thermalpref_mapping[x])
thermalpref_eval['Thermal Preference'] = thermalpref_eval['Thermal Preference'].apply(lambda x: thermalpref_mapping[x])
thermalpref_test['Thermal Preference'] = thermalpref_test['Thermal Preference'].apply(lambda x: thermalpref_mapping[x])

thermalacc_train['Thermal Acceptability'] = thermalacc_train['Thermal Acceptability'].apply(lambda x: thermalacc_mapping[x])
thermalacc_eval['Thermal Acceptability'] = thermalacc_eval['Thermal Acceptability'].apply(lambda x: thermalacc_mapping[x])
thermalacc_test['Thermal Acceptability'] = thermalacc_test['Thermal Acceptability'].apply(lambda x: thermalacc_mapping[x])

airpref_train['Air Movement Preference'] = airpref_train['Air Movement Preference'].apply(lambda x: airpref_mapping[x])
airpref_eval['Air Movement Preference'] = airpref_eval['Air Movement Preference'].apply(lambda x: airpref_mapping[x])
airpref_test['Air Movement Preference'] = airpref_test['Air Movement Preference'].apply(lambda x: airpref_mapping[x])

# Define model architecture

In [23]:
def define_model_architecture(num_classes):
    """
    Defines the CNN LSTM model architecture with model parameters.
    
    Parameters:
        num_classes: An integer value indicating the number of output classes for the model.
        
    Return:
        model: The Keras object containing the CNN LSTM model architecture.
    """

    # Create a Sequential model
    model = Sequential()

    # 1D Convolutional Layer (Part of the model to be retrained)
    model.add(Conv1D(filters=128, kernel_size=5, padding='same', input_shape=(args.input_size, 1), trainable=True))
    model.add(Dropout(0.1))

    # Two LSTM Layers (Part of the model to be retrained)
    model.add(LSTM(256, return_sequences=True, recurrent_dropout=0.1, trainable=True))
    model.add(LSTM(256, return_sequences=False, recurrent_dropout=0.1, trainable=True))

    # Flatten the output from LSTM layers
    model.add(Flatten())

    # Two Dense (Fully Connected) Layers (These layers will remain fixed)
    model.add(Dense(64, activation='relu', kernel_initializer='glorot_uniform', trainable=True))
    model.add(Dense(16, activation='relu', kernel_initializer='glorot_uniform', trainable=True))

    # Output Layer (Part of the model to be retrained)
    model.add(Dense(num_classes, activation='softmax', trainable=True))

    # Compile the model
    optimizer = tf.keras.optimizers.Adam(learning_rate=args.learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    # Print model summary
    print(model.summary())
    
    return model

# Model training and evaluation

In [25]:
def train_model(model, train_df, eval_df, model_name, target_col, num_classes):
    # Assuming train_df has columns for features and a 'target' column for labels
    X_train = np.array(train_df[args.input_features])
    y_train = np.array(train_df[target_col])
    
    X_val = np.array(eval_df[args.input_features])
    y_val = np.array(eval_df[target_col])

    # Create and fit a Min-Max scaler
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)

    # Convert labels to one-hot encoding
    y_train = tf.keras.utils.to_categorical(y_train, num_classes=num_classes)
    y_val = tf.keras.utils.to_categorical(y_val, num_classes=num_classes)

    # Define a callback to save the best model during training
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint('model.h5', save_best_only=True)
    
    # Early stopping callback
    early_stopping_callback = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',  # Monitor validation loss
        patience=5,          # Number of epochs with no improvement after which training will be stopped
        restore_best_weights=True  # Restore the best model weights when training stops
    )

    # Train the model
    history = model.fit(X_train_scaled, 
                        y_train, 
                        epochs=args.num_epochs, 
                        batch_size=args.batch_size,
                        validation_data=(X_val_scaled, y_val), 
                        callbacks=[checkpoint_callback, early_stopping_callback])

    # Save the final trained model
    model.save(model_name + '.h5')
    
    print(history)

    return model, scaler

def evaluate_model(model, test_df, scaler, model_name, target_col, metrics=['accuracy', 'weighted_f1']):
    # Assuming test_df has columns for features and a 'target' column for labels
    X_test = np.array(test_df[args.input_features])
    y_true = np.array(test_df[target_col])

    # Load the pre-trained model
    loaded_model = tf.keras.models.load_model(model_name + '.h5')
    
    # Apply numerical scaler on X_test
    X_test_scaled = scaler.transform(X_test)

    # Get predictions from the model
    y_pred = loaded_model.predict(X_test_scaled)
    y_pred_classes = np.argmax(y_pred, axis=1)

    evaluation_results = {}

    if 'accuracy' in metrics:
        accuracy = accuracy_score(y_true, y_pred_classes)
        evaluation_results['accuracy'] = accuracy

    if 'weighted_f1' in metrics:
        weighted_f1 = f1_score(y_true, y_pred_classes, average='weighted')
        evaluation_results['weighted_f1'] = weighted_f1

    return evaluation_results


In [11]:
# def train_model(model, train_df, model_name, target_col, num_classes):
#     # Assuming train_df has columns for features and a 'target' column for labels
#     X = np.array(train_df[args.input_features])
#     y = np.array(train_df[target_col])

#     # Split the data into training and validation sets
#     X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
    
#     # Create and fit a Min-Max scaler
#     scaler = MinMaxScaler()
#     X_train_scaled = scaler.fit_transform(X_train)
#     X_val_scaled = scaler.transform(X_val)

#     # Convert labels to one-hot encoding
#     y_train = tf.keras.utils.to_categorical(y_train, num_classes=num_classes)
#     y_val = tf.keras.utils.to_categorical(y_val, num_classes=num_classes)

#     # Define a callback to save the best model during training
#     checkpoint_callback = tf.keras.callbacks.ModelCheckpoint('model.h5', save_best_only=True)
    
#     # Early stopping callback
#     early_stopping_callback = tf.keras.callbacks.EarlyStopping(
#         monitor='val_loss',  # Monitor validation loss
#         patience=5,          # Number of epochs with no improvement after which training will be stopped
#         restore_best_weights=True  # Restore the best model weights when training stops
#     )

#     # Train the model
#     history = model.fit(X_train_scaled, 
#                         y_train, 
#                         epochs=args.num_epochs, 
#                         batch_size=args.batch_size,
#                         validation_data=(X_val_scaled, y_val), 
#                         callbacks=[checkpoint_callback])

#     # Save the final trained model
#     model.save(model_name + '.h5')
    
#     print(history)

#     return model, scaler

# def evaluate_model(model, test_df, scaler, model_name, target_col, metrics=['accuracy', 'weighted_f1']):
#     # Assuming test_df has columns for features and a 'target' column for labels
#     X_test = np.array(test_df[args.input_features])
#     y_true = np.array(test_df[target_col])

#     # Load the pre-trained model
#     loaded_model = tf.keras.models.load_model(model_name + '.h5')
    
#     # Apply numerical scaler on X_test
#     X_test_scaled = scaler.transform(X_test)

#     # Get predictions from the model
#     y_pred = loaded_model.predict(X_test_scaled)
#     y_pred_classes = np.argmax(y_pred, axis=1)

#     evaluation_results = {}

#     if 'accuracy' in metrics:
#         accuracy = accuracy_score(y_true, y_pred_classes)
#         evaluation_results['accuracy'] = accuracy

#     if 'weighted_f1' in metrics:
#         weighted_f1 = f1_score(y_true, y_pred_classes, average='weighted')
#         evaluation_results['weighted_f1'] = weighted_f1

#     return evaluation_results


In [12]:
# # Train and evaluate thermal preference model (source: ASHRAE, target: ASHRAE)
# print("Training Thermal Preference Model")
# thermalpref_model = define_model_architecture(num_classes=3)
# thermalpref_model, thermalpref_scaler = train_model(thermalpref_model, 
#                                                     thermalpref_train, 
#                                                     model_name='cnnlstm_base_thermalpref_model', 
#                                                     target_col='Thermal Preference', 
#                                                     num_classes=3)

# thermalpref_eval = evaluate_model(thermalpref_model, 
#                                   thermalpref_test, 
#                                   thermalpref_scaler,
#                                   model_name='cnnlstm_base_thermalpref_model', 
#                                   target_col='Thermal Preference')
# print("Thermal Preference Accuracy:", thermalpref_eval['accuracy'])
# print("Thermal Preference Weighted F1 Score:", thermalpref_eval['weighted_f1'])

Training Thermal Preference Model


2024-08-11 13:10:30.177282: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-08-11 13:10:30.177728: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2024-08-11 13:10:30.177752: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (7bde9dfab31b): /proc/driver/nvidia/version does not exist
2024-08-11 13:10:30.178731: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 7, 128)            768       
                                                                 
 dropout (Dropout)           (None, 7, 128)            0         
                                                                 
 lstm (LSTM)                 (None, 7, 256)            394240    
                                                                 
 lstm_1 (LSTM)               (None, 256)               525312    
                                                                 
 flatten (Flatten)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 64)                16448     
                                                                 
 dense_1 (Dense)             (None, 16)                1

In [ ]:
# Train and evaluate thermal preference model (source: ASHRAE, target: ASHRAE)
print("Training Thermal Preference Model")

# Define the model architecture
thermalpref_model = define_model_architecture(num_classes=3)

# Train the model using the training and evaluation datasets
thermalpref_model, thermalpref_scaler = train_model(thermalpref_model, 
                                                    train_df=thermalpref_train, 
                                                    eval_df=thermalpref_eval, 
                                                    model_name='cnnlstm_base_thermalpref_model', 
                                                    target_col='Thermal Preference', 
                                                    num_classes=3)

# Evaluate the model using the test dataset
thermalpref_eval_results = evaluate_model(thermalpref_model, 
                                          test_df=thermalpref_test, 
                                          scaler=thermalpref_scaler,
                                          model_name='cnnlstm_base_thermalpref_model', 
                                          target_col='Thermal Preference')

# Print the evaluation results
print("Thermal Preference Accuracy:", thermalpref_eval_results['accuracy'])
print("Thermal Preference Weighted F1 Score:", thermalpref_eval_results['weighted_f1'])


Training Thermal Preference Model
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_2 (Conv1D)           (None, 4, 128)            768       
                                                                 
 dropout_2 (Dropout)         (None, 4, 128)            0         
                                                                 
 lstm_4 (LSTM)               (None, 4, 256)            394240    
                                                                 
 lstm_5 (LSTM)               (None, 256)               525312    
                                                                 
 flatten_2 (Flatten)         (None, 256)               0         
                                                                 
 dense_6 (Dense)             (None, 64)                16448     
                                                                 
 dense_7 (Dense)    

In [13]:
# Train and evaluate thermal acceptability model (source: ASHRAE, target: BCA)
print("Training Thermal Acceptability Model")
thermalacc_model = define_model_architecture(num_classes=2)
thermalacc_model, thermalacc_scaler = train_model(thermalacc_model, 
                                                  thermalacc_train, 
                                                  model_name='cnnlstm_base_thermalacc_model', 
                                                  target_col='Thermal Acceptability', 
                                                  num_classes=2)

thermalacc_eval = evaluate_model(thermalacc_model, 
                                 thermalacc_test, 
                                 thermalacc_scaler,
                                 model_name='cnnlstm_base_thermalacc_model', 
                                 target_col='Thermal Acceptability')
print("Thermal Acceptability Accuracy:", thermalacc_eval['accuracy'])
print("Thermal Acceptability Weighted F1 Score:", thermalacc_eval['weighted_f1'])

Training Thermal Acceptability Model
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_1 (Conv1D)           (None, 7, 128)            768       
                                                                 
 dropout_1 (Dropout)         (None, 7, 128)            0         
                                                                 
 lstm_2 (LSTM)               (None, 7, 256)            394240    
                                                                 
 lstm_3 (LSTM)               (None, 256)               525312    
                                                                 
 flatten_1 (Flatten)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 64)                16448     
                                                                 
 dense_4 (Dense) 

In [14]:
# Train and evaluate air movement preference model (source: ASHRAE, target: BCA)
print("Training Air Movement Preference Model")
airpref_model = define_model_architecture(num_classes=3)
airpref_model, airpref_scaler = train_model(airpref_model, 
                                            airpref_train, 
                                            model_name='cnnlstm_base_airpref_model', 
                                            target_col='Air Movement Preference', 
                                            num_classes=3)

airpref_eval = evaluate_model(airpref_model, 
                              airpref_test, 
                              airpref_scaler,
                              model_name='cnnlstm_base_airpref_model', 
                              target_col='Air Movement Preference')
print("Air Movement Preference Accuracy:", airpref_eval['accuracy'])
print("Air Movement Preference Weighted F1 Score:", airpref_eval['weighted_f1'])

Training Air Movement Preference Model
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_2 (Conv1D)           (None, 7, 128)            768       
                                                                 
 dropout_2 (Dropout)         (None, 7, 128)            0         
                                                                 
 lstm_4 (LSTM)               (None, 7, 256)            394240    
                                                                 
 lstm_5 (LSTM)               (None, 256)               525312    
                                                                 
 flatten_2 (Flatten)         (None, 256)               0         
                                                                 
 dense_6 (Dense)             (None, 64)                16448     
                                                                 
 dense_7 (Dense